<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
اضافه‌کردن ویژگی‌های جدید
</font>
</h1>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
مجموعه‌داده‌ی خانه‌های شهر پکن، اطلاعات بسیار مفیدی در اختیار ما قرار می‌دهد، اما گاهی اوقات می‌توان با استفاده از ویژگی‌های موجود، ویژگی‌های جدیدی هم تولید کرد. برای مثال، با داشتن متراژ و قیمت کلی هر خانه، می‌توانیم برای هر خانه، قیمت هر متر مربع را حساب کنیم که یک ویژگی مهم در تحلیل اطلاعات مسکن می‌باشد. در این قسمت از پروژه دو ویژگی جدید به مجموعه‌داده خود اضافه می‌کنیم و پس از آن با ابزار‌های مصورسازی، شروع به رسم نمودار‌هایی برای بررسی روابط بین آن‌ها می‌کنیم.

</font>
</p>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
ابتدا فایل مجموعه‌داده‌ی <code>housing_no_outlier_full.csv</code> را می‌خوانیم.
</font>
</p>


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

housing_no_outlier = pd.read_csv('housing_no_outlier.csv', encoding='gbk')

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت اول
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
یکی از کارهای جذابی که در چنین تحلیل‌هایی می‌توان انجام داد، ساخت ویژگی جدید بر اساس ویژگی‌های موجود می‌باشد. البته این کار به‌شرطی باید انجام شود که ویژگی جدید، ویژگی مفیدی محسوب شود.
 به‌عنوان اولین قدم از شما می‌خواهیم فاصله‌ی هر خانه تا مرکز پایتخت چین را محاسبه کنید و در ستونی به اسم <code>distanceToCapital</code> ذخیره کنید.
    طول جغرافیایی (<code>longitude</code>) و عرض جغرافیایی (<code>latitude</code>) مرکز پایتخت چین به ترتیب <code>116.4074</code> و <code>39.9042</code> می‌باشند.
    برای محاسبه‌ی فاصله می‌توانید از فرمولی که در پایین آمده است استفاده کنید.
  دقت کنید که در محاسبات، باید همه‌ی زاویه‌ها را به رادیان تبدیل کنید. نام دیتافریم جدید که شامل این ستون است را <code>housing_capital</code> قرار دهید.
<br>
<br>
<span style="color:green"><b>راهنمایی:</b></span>
محاسبات را با نامپای انجام دهید.
</font>
</p>


$$ distance = acos(sinφ_1*sinφ_2 + cosφ_1*cosφ_2 * cos(λ_2-λ_1)) * 6371.0088 $$
$$ φ: \text{latitude} $$ 
$$ λ: \text{longitude} $$ 

In [3]:
from math import radians, cos, sin, acos

capital_Lng = radians(116.4074)
capital_Lat = radians(39.9042)

housing_capital = housing_no_outlier.copy()

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0088  # Earth radius in kilometers
    d_lat = np.radians(lat2 - lat1)
    d_lon = np.radians(lon2 - lon1)
    a = np.sin(d_lat/2) * np.sin(d_lat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(d_lon/2) * np.sin(d_lon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance

housing_capital['distanceToCapital'] = housing_capital.apply(lambda row: haversine(row['Lat'], row['Lng'], 39.9042, 116.4074), axis=1)

housing_capital.head()


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت دوم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این مجموعه‌داده، قیمت معامله‌شده و متراژ هر خانه مشخص است. اما قیمت هر متر مربع خانه مشخص نیست. در ستون <code>pricePerSquare</code> قیمت هر متر مربع خانه را محاسبه و ذخیره کنید. توجه کنید که قیمت کلی بر اساس <i>میلیون یوان</i> است. پس قیمتی که برای هر متر مربع از خانه حساب می‌کنید، باید در هزار ضرب شود تا قیمت بر اساس <i>یوان</i> به‌دست آید. نام دیتافریم حاصل را برابر <code>housing_PPS</code> قرار دهید.
</font>
</p>


In [4]:
housing_PPS = housing_capital.copy()

housing_PPS['pricePerSquare'] = (housing_PPS["totalPrice"] * 1000 )/ housing_PPS["square"]
housing_PPS.head()


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
می‌خواهیم با مصور‌سازی داده به روش‌های گوناگون، اطلاعاتی متناسب با نیاز های خود استخراج کنیم. مجموعه‌داده‌ی ما نه مقدار گم‌شده دارد، نه داده‌ی پرت دارد، و نه ستون‌هایی که با فرمت نامناسب ذخیره شده‌ باشند. در اولین قدم نگاهی به هیستوگرام ستون های عددی می‌اندازیم.
<br>
برای داشتن جدولی از هیستوگرام همه‌ی ستون‌های عددی، باید از تابع <code>hist</code> ‌استفاده کنیم. دقت کنید که تابع <code>hist</code> با چیزی که در آموزش‌های مربوط به رابط پانداز دیده بودیم فرق دارد. <code>plot(kint = ‘hist’)</code> هیستوگرام همه‌ی ستون‌ها را بر روی یک صفحه‌محور می‌کشید اما تابع <code>hist</code> برای هر ستون یک صفحه‌محور جداگانه رسم می‌کند.
</font>
</p>

In [5]:
fig , ax = plt.subplots()
fig.set_size_inches(15,15)
housing_PPS.hist(ax = ax, bins = 20);

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
با توجه به نمودار بالا، قیمت خانه یک خمیدگی به سمت قیمت ۲۰۰ میلیون یوان دارد. اکثر خانه‌ها حوالی سال ۲۰۰۰ ساخته شده‌اند. بیشترین فاصله از مرکز پایتخت ۴۰ کیلومتر است. اکثر خانه‌ها دو اتاق هال دارند. می‌توان اطلاعات بیشتری از این نمودار بیرون کشید.
</font>
</p>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
بررسی تاثیر فاصله از مرکز بر روی قیمت‌ هر متر مربع</font>
</h2>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
می‌خواهیم از ستون جدیدی که ساختیم استفاده کنیم و ببینیم که با دورشدن از مرکز شهر،‌ قیمت هر متر مربع از خانه چگونه تغییر می‌کند. برای این‌کار می توان از نمودار <code>regplot</code>  سیبورن استفاده کرد که علاوه بر نمایش نقاط،‌ با رگرسیون روند داده را نشان می‌دهد.
(اجرای سلول زیر ممکن است چند دقیقه طول بکشد.)
</font>
</p>

In [6]:
fig , ax = plt.subplots()
fig.set_size_inches(8,8)

sns.regplot(ax = ax, data = housing_PPS, x = 'distanceToCapital', y = 'pricePerSquare', 
                   line_kws={'color':'red'}, scatter_kws= {'alpha': 0.1});

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت سوم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این بخش می‌خواهیم بررسی کنیم که داشتن آسانسور تا چه‌اندازه بر
<b>
 قیمت هر متر مربع
</b>
  خانه‌ها تاثیر می‌گذارد. برای این کار از نمودار‌های مختلفی می‌توان استفاده کرد. برای مثال با کمک نمودار میله‌ای می‌توان میانگین قیمت  خانه‌هایی که آسانسور دارند و آسانسور ندارند را مقایسه کرد یا با کمک نمودار جعبه‌ای می‌توان نموداری کشید که توزیع قیمت آن‌ها را نشان دهد. ما از نمودار چگالی برای مقایسه استفاده می‌کنیم.
<br>
<ul style="direction: rtl;text-align: right;">
  <li>
ابتدا یک شکل شامل یک صفحه‌محور بسازید و اندازه‌ی طول و عرض شکل را برابر ۶ اینچ قرار دهید.
</li>
  <li>
با استفاده از رابط پانداز، نمودار تخمین توزیع قیمت را برای خانه‌های بدون آسانسور رسم کنید. این نمودار را بر روی صفحه‌محوری که در مرحله‌ی قبل ساختید رسم کنید.
</li>
  <li>
سپس نمودار تخمین توزیع قیمت را برای خانه‌های دارای آسانسور روی همان صفحه‌محور رسم کنید.
</li>


  <li>
تنظیمات را طوری تغییر دهید که خروجی شما دقیقا مانند شکل زیر شود. رنگ‌های استفاده شده:
<ul>
<li>
رنگ عنوان شکل:‌ <code>red</code>
</li>
<li>
رنگ زمینه صفحه‌محور: <code>#ffffcc</code>
</li>
<li>
رنگ زمینه شکل: <code>khaki</code>
</li>
</ul>
</font>
</p>

<p style="text-align:center;"><img src="output.png"></p>

In [7]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(6,6))
housing_PPS[housing_PPS['elevator'] == 'no elevator']['pricePerSquare'].plot(kind='kde', label='no elevator', color='blue')
housing_PPS[housing_PPS['elevator'] == 'has elevator']['pricePerSquare'].plot(kind='kde', label='has elevator', color='orange')
ax.set_title("effect of elevator on price",color="red")
ax.set_facecolor('#ffffcc')
fig.set_facecolor('khaki')
ax.set_xlabel("pricePerSquare")

plt.legend()
plt.show()

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
دیتافریم نهایی را برای استفاده در مراحل بعدی پروژه در فایلی به‌نام <code>housing_extended.csv</code> ذخیره می‌کنیم.
</font>
</p>

In [8]:
housing_PPS.to_csv('housing_extended.csv', encoding='gbk', index=False)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>


In [ ]:
import zipfile
import joblib
import nbformat as nbf

ntbk = nbf.read("3_add_new_features.ipynb", nbf.NO_CONVERT)
new_ntbk = ntbk
for cell in ntbk.cells:
    cell['outputs']=[]
nbf.write(new_ntbk, "3_add_new_features_no_output.ipynb", version=nbf.NO_CONVERT)

housing_capital.head(100).to_csv("housing_capital.csv") 
housing_PPS.head(100).to_csv("housing_PPS.csv")
joblib.dump(fig, "fig")

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

file_names = ["housing_capital.csv", "housing_PPS.csv", "fig", '3_add_new_features_no_output.ipynb']
compress(file_names)